In [7]:
import numpy as np
import pickle
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import os
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn import *
from sklearn.metrics import *
sns.set()
import warnings
warnings.filterwarnings("ignore")
length = 277


In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16223728225098454317
xla_global_id: -1
]


In [9]:
train_values = pd.read_csv('./data/train_test/train_patients.csv').values
test_values = pd.read_csv('./data/train_test/test_patients.csv').values

print(train_values.shape)
print(test_values.shape)


# Separate the training and testing data, and one-hot encode Y:
X_train = train_values[:,:-2]
X_test = test_values[:,:-2]
y_train = train_values[:,-2]
y_test = test_values[:,-2]
X_test1 = X_test.reshape(-1, X_train.shape[1], 1)
y_test1=to_categorical(y_test)


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42,stratify=y_train)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
X_train1 = X_train.reshape(-1, X_train.shape[1], 1)
y_train1=to_categorical(y_train)

X_val1 = X_val.reshape(-1, X_train.shape[1], 1)
y_val1=to_categorical(y_val)


(16607, 277)
(67116, 277)
X_train shape: (13285, 275)
X_val shape: (3322, 275)
y_train shape: (13285,)
y_val shape: (3322,)


In [10]:
import optuna
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def objective(trial):
    # Define the search space for hyperparameters
    num_layers = trial.suggest_int('num_layers', 1, 2)
    layer_size = trial.suggest_int('layer_size', 50, 800)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.2)
    use_batch_norm = trial.suggest_categorical('use_batch_norm', [True, False])
    activation = trial.suggest_categorical('activation', ['relu','tanh'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-3)

    # Create the MLP model with hyperparameters
    mlpmodel = Sequential()
    mlpmodel.add(Dense(layer_size, input_shape=(X_train.shape[1],)))
    mlpmodel.add(Dropout(dropout_rate))
    
    for _ in range(num_layers):
        mlpmodel.add(Dense(layer_size))
        if use_batch_norm:
            mlpmodel.add(BatchNormalization())
        mlpmodel.add(Activation(activation))
        mlpmodel.add(Dropout(dropout_rate))
    
    mlpmodel.add(Dense(7, activation='softmax'))

    # Compile the model with hyperparameters
    mlpmodel.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model with hyperparameters
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min', min_delta=0.001, restore_best_weights=True)
    pruning_callback = optuna.integration.TFKerasPruningCallback(trial, 'val_loss')
    history = mlpmodel.fit(X_train, y_train1, validation_data=(X_val, y_val1), epochs=50, callbacks=[early_stopping, pruning_callback], verbose=1)
    
    # Return the validation accuracy as the objective value
    return history.history['val_accuracy'][-1]

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and objective value
best_params = study.best_params
best_value = study.best_value
print('Best Hyperparameters:', best_params)
print('Best Objective Value:', best_value)


[I 2024-03-19 19:58:44,230] A new study created in memory with name: no-name-c51a1804-8ab7-47a4-92aa-8cd47c366d64


Epoch 1/50
416/416 [==============================] - 2s 4ms/step - loss: 0.5551 - accuracy: 0.8263 - val_loss: 0.3109 - val_accuracy: 0.9064
Epoch 2/50
416/416 [==============================] - 1s 3ms/step - loss: 0.3526 - accuracy: 0.8921 - val_loss: 0.2572 - val_accuracy: 0.9118
Epoch 3/50
416/416 [==============================] - 1s 3ms/step - loss: 0.2795 - accuracy: 0.9148 - val_loss: 0.1871 - val_accuracy: 0.9422
Epoch 4/50
416/416 [==============================] - 1s 3ms/step - loss: 0.2470 - accuracy: 0.9221 - val_loss: 0.1730 - val_accuracy: 0.9449
Epoch 5/50
416/416 [==============================] - 1s 3ms/step - loss: 0.2119 - accuracy: 0.9329 - val_loss: 0.1507 - val_accuracy: 0.9533
Epoch 6/50
416/416 [==============================] - 1s 3ms/step - loss: 0.1881 - accuracy: 0.9368 - val_loss: 0.1177 - val_accuracy: 0.9642
Epoch 7/50
416/416 [==============================] - 1s 3ms/step - loss: 0.1735 - accuracy: 0.9429 - val_loss: 0.1078 - val_accuracy: 0.9660
Epoch 

[I 2024-03-19 19:59:35,693] Trial 0 finished with value: 0.9783263206481934 and parameters: {'num_layers': 2, 'layer_size': 172, 'dropout_rate': 0.1248062459518126, 'use_batch_norm': True, 'activation': 'tanh', 'learning_rate': 0.0006269130974735595}. Best is trial 0 with value: 0.9783263206481934.


Epoch 1/50
416/416 [==============================] - 2s 3ms/step - loss: 0.6375 - accuracy: 0.7911 - val_loss: 0.2680 - val_accuracy: 0.9163
Epoch 2/50
416/416 [==============================] - 1s 3ms/step - loss: 0.3398 - accuracy: 0.8924 - val_loss: 0.1870 - val_accuracy: 0.9416
Epoch 3/50
416/416 [==============================] - 1s 3ms/step - loss: 0.2781 - accuracy: 0.9090 - val_loss: 0.1632 - val_accuracy: 0.9449
Epoch 4/50
416/416 [==============================] - 1s 3ms/step - loss: 0.2432 - accuracy: 0.9195 - val_loss: 0.1377 - val_accuracy: 0.9530
Epoch 5/50
416/416 [==============================] - 1s 3ms/step - loss: 0.2200 - accuracy: 0.9274 - val_loss: 0.1292 - val_accuracy: 0.9591
Epoch 6/50
416/416 [==============================] - 1s 3ms/step - loss: 0.2042 - accuracy: 0.9328 - val_loss: 0.1184 - val_accuracy: 0.9594
Epoch 7/50
416/416 [==============================] - 1s 3ms/step - loss: 0.1939 - accuracy: 0.9374 - val_loss: 0.1175 - val_accuracy: 0.9600
Epoch 

[I 2024-03-19 20:00:21,858] Trial 1 finished with value: 0.9813365340232849 and parameters: {'num_layers': 2, 'layer_size': 133, 'dropout_rate': 0.18550895913745297, 'use_batch_norm': True, 'activation': 'relu', 'learning_rate': 0.0005097814609911406}. Best is trial 1 with value: 0.9813365340232849.


...

Epoch 1/50
416/416 [==============================] - 2s 4ms/step - loss: 0.5975 - accuracy: 0.8072 - val_loss: 0.3549 - val_accuracy: 0.8865
Epoch 2/50
416/416 [==============================] - 1s 3ms/step - loss: 0.3419 - accuracy: 0.8973 - val_loss: 0.2727 - val_accuracy: 0.9118
Epoch 3/50
416/416 [==============================] - 1s 3ms/step - loss: 0.2706 - accuracy: 0.9147 - val_loss: 0.2225 - val_accuracy: 0.9272
Epoch 4/50
416/416 [==============================] - 1s 3ms/step - loss: 0.2197 - accuracy: 0.9315 - val_loss: 0.1869 - val_accuracy: 0.9347
Epoch 5/50
416/416 [==============================] - 1s 3ms/step - loss: 0.1870 - accuracy: 0.9421 - val_loss: 0.1577 - val_accuracy: 0.9521
Epoch 6/50
416/416 [==============================] - 1s 3ms/step - loss: 0.1603 - accuracy: 0.9536 - val_loss: 0.1391 - val_accuracy: 0.9512
Epoch 7/50
416/416 [==============================] - 1s 3ms/step - loss: 0.1385 - accuracy: 0.9594 - val_loss: 0.1361 - val_accuracy: 0.9564
Epoch 

[I 2024-03-19 20:37:04,440] Trial 98 finished with value: 0.9864539504051208 and parameters: {'num_layers': 1, 'layer_size': 410, 'dropout_rate': 0.10324976064678713, 'use_batch_norm': False, 'activation': 'tanh', 'learning_rate': 0.00011326523260917271}. Best is trial 10 with value: 0.987959086894989.


Epoch 1/50
413/416 [============================>.] - ETA: 0s - loss: 0.5900 - accuracy: 0.8142

[I 2024-03-19 20:37:07,370] Trial 99 pruned. Trial was pruned at epoch 0.


Best Hyperparameters: {'num_layers': 1, 'layer_size': 763, 'dropout_rate': 0.13714949392899192, 'use_batch_norm': False, 'activation': 'tanh', 'learning_rate': 0.00010157391045421977}
Best Objective Value: 0.987959086894989


In [11]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

In [16]:
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour
from optuna.visualization import plot_intermediate_values

In [17]:
plot_intermediate_values(study)

In [18]:
plot_contour(study)

In [21]:
plot_param_importances(study)

In [24]:

# Get the best trial
best_trial = study.best_trial
best_trial


FrozenTrial(number=10, state=TrialState.COMPLETE, values=[0.987959086894989], datetime_start=datetime.datetime(2024, 3, 19, 20, 2, 52, 371314), datetime_complete=datetime.datetime(2024, 3, 19, 20, 3, 43, 204681), params={'num_layers': 1, 'layer_size': 763, 'dropout_rate': 0.13714949392899192, 'use_batch_norm': False, 'activation': 'tanh', 'learning_rate': 0.00010157391045421977}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.3238540589809418, 1: 0.2448163479566574, 2: 0.18906839191913605, 3: 0.1632336676120758, 4: 0.14054667949676514, 5: 0.12353552132844925, 6: 0.10443636029958725, 7: 0.09346316009759903, 8: 0.08571696281433105, 9: 0.07661384344100952, 10: 0.07303795218467712, 11: 0.07291659712791443, 12: 0.06954682618379593, 13: 0.062357641756534576, 14: 0.06531557440757751, 15: 0.05640150606632233, 16: 0.05698106437921524, 17: 0.05346554145216942, 18: 0.05226834863424301, 19: 0.05086370185017586, 20: 0.0502149723470211, 21: 0.051497992128133774, 22: 0.049759894609451294, 

In [12]:
class_counts = np.bincount(y_test.astype(int))
print(class_counts)


[    0 54729  4075  3746  2328  1850   388]


In [20]:
from sklearn.metrics import confusion_matrix, f1_score

# Predict the test data
y_pred = mlp.predict(X_test1)

# Convert the predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Create the confusion matrix
cm = confusion_matrix(y_test, y_pred_labels)
print("Confusion Matrix:")
print(cm)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred_labels, average='weighted')
print("F1 Score:", f1)




NameError: name 'mlp' is not defined